In [28]:
import os    
import pandas as pd
import ollama
import chromadb

In [29]:
def parse_abstract(text):
    abs = text.split("\\\\")[2].strip()
    return abs


In [30]:
ids = []
abstracts = []
years = ["1992","1993","1994"]
cnt = 0

In [31]:
for folder in os.listdir('cit-HepTh-abstracts'):
    if folder==".DS_Store":
        continue
    # if folder not in years:
    #     continue
    # if cnt > 10:
    #     break
    for file in os.listdir('cit-HepTh-abstracts/'+folder):
        with open('cit-HepTh-abstracts/'+folder+'/'+file) as f:
            ids.append(file.split(".")[0])
            abstracts.append(parse_abstract(f.read()))
            # cnt += 1
            # if cnt > 10:
            #     break


In [37]:
client = chromadb.PersistentClient(path="full_data/")
collection = client.create_collection(name="docs")

# store each document in a vector embedding database
for i, d in enumerate(abstracts):
  response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[ids[i]],
    embeddings=[embedding],
    documents=[d]
  )
  


In [42]:
len(abstracts)

29555

In [43]:
# an example prompt
prompt = "quantum"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="mxbai-embed-large"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=10
)
data = results['documents'][0][0]
data

'A quantum theory is developed for a difference-difference system which can\nserve as a toy-model of the quantum Korteveg-de-Vries equation.'

In [44]:
results

{'ids': [['9509024',
   '9305001',
   '9410168',
   '9304147',
   '0112076',
   '9411196',
   '9402062',
   '9304036',
   '9308061',
   '9808047']],
 'embeddings': None,
 'documents': [['A quantum theory is developed for a difference-difference system which can\nserve as a toy-model of the quantum Korteveg-de-Vries equation.',
   'It is shown that it is possible to measure the Schr\\"odinger wave of a single\nquantum system. This provides a strong argument for associating physical\nreality with a quantum state of a single system in sharp contrast with the\nusual approach in which the physical meaning of a quantum state is related only\nto an ensemble of identical systems. An apparent paradox between measurability\nof a quantum state of a single system and the relativistic causality is\nresolved.',
   'Quantum measurement theory has fallen under the resticting influence of the\nattempt to explain the fundamental axioms of quantum theory in terms of the\ntheory itself. This has not only 